# Generate new molecules based on the public skeleton GAN prediction QM9 data set

## 项目背景 
在过去的几年里，深度学习技术在分子生成方面取得了巨大的进步。许多研究人员开发了基于深度学习的分子生成模型，例如基于图神经网络的分子生成模型（Graph-based Neural Network, GNN-based molecule generator）。然而，这些模型往往需要大量的训练数据才能取得良好的性能。为了解决这个问题，最近的研究人员提出了一种新的分子生成方法，即基于公共骨架的GAN（Generative Adversarial Network）预测QM9数据集。该方法利用GAN生成的分子骨架，并在骨架上添加新的取代基，生成新的分子。这种方法比直接使用GAN生成分子更可控，也更容易满足指定的骨架结构。

本项目的目标是利用公共骨架的GAN预测QM9数据集，生成符合Lipinski's Rule of Five和CNS药物设计规则的新分子。我们将使用PyTorch和RDKit库来实现该项目。

## 设计思想 ：

1. **加载SDF文件中的分子**：从指定的SDF文件中加载具有最大公共骨架的分子，以作为新分子生成的起点。
2. **生成新分子**：使用反应模板，比如：xxxx, 在骨架上添加新的取代基，生成新的分子。这种方法比直接使用GAN生成分子更可控，也更容易满足指定的骨架结构。
3. **应用药物设计规则和计算分子属性**：对生成的新分子，应用Lipinski's Rule of Five和CNS药物设计规则，筛选出符合条件的分子。同时计算额外的分子属性，以供进一步分析。
4. **使用GNN模型预测分子性质**：将生成的新分子转换为图数据，并使用之前训练的GNN模型预测其性质（例如目标物理化学性质或生物活性）。
5. **筛选符合要求的分子**：根据预测的性质值、QED得分、Lipinski规则和CNS药物设计规则，筛选出符合要求的分子。
6. **并行计算**：在处理大量分子时，使用多线程或并行计算来加速处理。
7. **保存结果**：将筛选后的分子及其属性保存到CSV文件中。

## 1. 导入依赖库

In [1]:
import os
from pathlib import Path
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, QED, rdChemReactions
from rdkit.Chem import rdFMCS
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
import random
# pip install deap
from deap import base, creator, tools, algorithms
# 检查安装，安装完成后，你可以通过以下命令来检查是否成功安装：
# python -c "import deap; print(deap.__version__)"

# pip install joblib：用于并行化操作
from joblib import Parallel, delayed
import multiprocessing

In [2]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)


/Users/wangyang/Desktop/AIDD/09_Generate_new_molecules_based_on_the_public_skeleton_GAN_prediction_QM9_data_set/data


In [3]:
# 加载最大的骨架分子
sdf_file = str(HERE / "../05_Compound_clustering/data/molecule_set_largest_cluster.sdf")
supplier = Chem.SDMolSupplier(sdf_file)
scaffold_molecules = [mol for mol in supplier if mol is not None]

print(f"Loaded {len(scaffold_molecules)} molecules from the SDF file.")

Loaded 108 molecules from the SDF file.


# =======================
# 第一步：训练GNN模型
# =======================

In [4]:
# 加载QM9数据集
from torch_geometric.datasets import QM9

In [ ]:

from torch_geometric.datasets import QM9
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool
path = DATA 
target_property = 0  # 选择要预测的属性（0-12）
dataset = QM9(root=DATA)
dataset[0]

Processing...


In [6]:
# 标准化目标属性
mean = dataset.data.y[:, target_property].mean().item()
std = dataset.data.y[:, target_property].std().item()
dataset.data.y[:, target_property] = (dataset.data.y[:, target_property] - mean) / std


AttributeError: 'DataFrame' object has no attribute 'data'

In [ ]:
# 划分训练集、验证集和测试集
torch.manual_seed(42)
dataset = dataset.shuffle()
train_dataset = dataset[:110000]
val_dataset = dataset[110000:120000]
test_dataset = dataset[120000:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# 定义GNN模型
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_features, 128)
        self.conv2 = GCNConv(128, 128)
        self.conv3 = GCNConv(128, 128)
        self.lin1 = nn.Linear(128, 64)
        self.lin2 = nn.Linear(64, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x.squeeze()


In [ ]:
# 创建模型实例
model = GCN()

In [ ]:
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 训练和验证函数
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        y = data.y[:, target_property]
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_dataset)

def validate(loader):
    model.eval()
    total_loss = 0
    preds = []
    targets = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            y = data.y[:, target_property]
            loss = criterion(out, y)
            total_loss += loss.item() * data.num_graphs
            preds.append(out.cpu().numpy())
            targets.append(y.cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    r2 = r2_score(targets, preds)
    rmse = mean_squared_error(targets, preds, squared=False)
    return total_loss / len(loader.dataset), r2, rmse

In [ ]:
# 开始训练
best_val_loss = float('inf')
for epoch in range(1, 51):
    train_loss = train()
    val_loss, val_r2, val_rmse = validate(val_loader)
    print(f'Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val R2: {val_r2:.4f}, Val RMSE: {val_rmse:.4f}')
    # 保存最好的模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# 在测试集上评估模型
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_r2, test_rmse = validate(test_loader)
print(f'Test Loss: {test_loss:.4f}, Test R2: {test_r2:.4f}, Test RMSE: {test_rmse:.4f}')


# =======================
# 第二步：生成新分子
# =======================

In [ ]:
# 从文献和数据库中扩展取代基库和反应模板
# 这里使用了一些示例，实际应用中可以从ChEMBL、ZINC等数据库获取
substituents = [
    'CC', 'CCC', 'CCCC', 'c1ccccc1', 'c1ccncc1', 'c1cccnc1',
    'CO', 'CN', 'Cl', 'Br', 'F', 'OC', 'NC', 'SC',
    'c1ccccc1C', 'c1ccccc1O', 'c1ccccc1N',
    'N#C', 'C#N', 'C=O', 'N=O', 'S(=O)(=O)N', 'C(=O)N',
    # 更多取代基
]

In [ ]:
# 多种反应模板
reaction_smarts_list = [
    # Suzuki反应
    '[c:1][Br:2].[B:3]([OH])[OH]>>[c:1][c:3]',
    # 酰胺化反应
    '[C:1](=[O])[Cl:2].[N:3]>>[C:1](=[O])[N:3]',
    # 醇化反应
    '[C:1][Br:2].[O:3][H]>>[C:1][O:3]',
    # 还原胺化
    '[C:1]=[O:2].[N:3][H2]>>[C:1][N:3]',
    # 其他反应模板
]

In [ ]:
reactions = [rdChemReactions.ReactionFromSmarts(s) for s in reaction_smarts_list]

In [ ]:
# 定义取代基的分子对象
substituent_mols = [Chem.MolFromSmiles(smi) for smi in substituents if Chem.MolFromSmiles(smi) is not None]


In [ ]:
# 使用并行计算生成新分子
def generate_products(scaffold_mol):
    products = []
    for substituent in substituent_mols:
        for reaction in reactions:
            reactants = (scaffold_mol, substituent)
            prods = reaction.RunReactants(reactants)
            for product_tuple in prods:
                product = product_tuple[0]
                try:
                    # 规范化产物
                    Chem.SanitizeMol(product)
                    products.append(product)
                except:
                    continue
    return products

In [ ]:
# 使用多线程并行生成分子
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(generate_products)(mol) for mol in scaffold_molecules)
new_molecules = [mol for sublist in results for mol in sublist]

print(f"Generated {len(new_molecules)} new molecules.")

In [ ]:
# 去除重复的分子
unique_molecules = {Chem.MolToSmiles(mol): mol for mol in new_molecules}
new_molecules = list(unique_molecules.values())

print(f"Unique molecules count: {len(new_molecules)}")

# =======================
# 第三步：优化生成策略（改进遗传算法）
# =======================

In [ ]:
# 定义适应度函数
def fitness_function(mol):
    try:
        data = mol_to_graph(mol)
        with torch.no_grad():
            pred_property = model(data).item()
        # 反标准化
        pred_property = pred_property * std + mean
        # 合成可行性评分（示例，这里简单使用QED评分）
        synth_score = QED.qed(mol)
        # 毒性预测（这里使用简单的规则，实际中可以使用更复杂的模型）
        toxic = any(substruct in Chem.MolToSmiles(mol) for substruct in ['[N+]', '[O-]'])
        if toxic:
            penalty = -1.0
        else:
            penalty = 0.0
        # 适应度为目标性质 + 合成可行性 - 毒性惩罚
        fitness = pred_property + synth_score + penalty
        return fitness,
    except:
        return -1000.0,

In [ ]:
# 初始化遗传算法工具箱
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # 最大化目标函数
creator.create("Individual", Chem.Mol, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", random.choice, new_molecules)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
# 改进交叉操作（基于化学规则）
def mate_mols(mol1, mol2):
    # 使用最大公共子结构（MCS）进行交叉
    try:
        mcs = rdFMCS.FindMCS([mol1, mol2])
        smarts = mcs.smartsString
        mcs_mol = Chem.MolFromSmarts(smarts)
        if mcs_mol is None:
            return mol1
        # 获取差异部分
        frags1 = Chem.ReplaceCore(mol1, mcs_mol)
        frags2 = Chem.ReplaceCore(mol2, mcs_mol)
        if frags1 is None or frags2 is None:
            return mol1
        # 随机选择一个片段进行组合
        new_mol = Chem.CombineMols(mcs_mol, random.choice([frags1, frags2]))
        new_mol = Chem.MolFromSmiles(Chem.MolToSmiles(new_mol))
        Chem.SanitizeMol(new_mol)
        return new_mol
    except:
        return mol1

In [ ]:
# 改进变异操作（替换取代基）
def mutate_mol(mol):
    try:
        # 随机选择一个取代基位置
        editable_mol = Chem.EditableMol(mol)
        atom_indices = [atom.GetIdx() for atom in mol.GetAtoms() if atom.GetAtomicNum() == 6]  # 碳原子
        if not atom_indices:
            return mol
        idx = random.choice(atom_indices)
        # 替换为随机取代基
        substituent = random.choice(substituent_mols)
        rxn = rdChemReactions.ReactionFromSmarts('[C:1]>>[C:1][*:2]')
        products = rxn.RunReactants((mol,))
        if products:
            product = products[0][0]
            Chem.SanitizeMol(product)
            return product
        else:
            return mol
    except:
        return mol

toolbox.register("mate", mate_mols)
toolbox.register("mutate", mutate_mol)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)


In [ ]:
# 运行遗传算法
population = toolbox.population(n=100)
NGEN = 20
for gen in range(NGEN):
    print(f"-- Generation {gen} --")
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    # 使用并行计算评估适应度
    fits = Parallel(n_jobs=num_cores)(delayed(toolbox.evaluate)(ind) for ind in offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

In [ ]:
# 获取最优分子
top_inds = tools.selBest(population, k=50)
optimized_molecules = [ind for ind in top_inds]

# =======================
# 第四步：应用药物设计规则和计算分子属性
# =======================

In [ ]:
from rdkit.Chem import Draw


In [ ]:
def lipinski_rule_of_five(mol):
    mol_weight = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    return mol_weight <= 500 and logp <= 5 and hbd <= 5 and hba <= 10

In [ ]:
# 定义CNS药物设计规则
def cns_drug_likeness(mol):
    mol_weight = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    tpsa = Descriptors.TPSA(mol)
    return mol_weight <= 400 and 2 <= logp <= 3 and tpsa <= 90

In [ ]:
# 其他评价指标
def synthetic_accessibility(mol):
    # 这里使用RDKit的SA_Score，需安装sascorer模块
    try:
        from rdkit.Chem import rdMolDescriptors
        sa_score = rdMolDescriptors._CalcSAScore(mol)
        return sa_score
    except:
        return None


In [ ]:
def toxicity_prediction(mol):
    # 简单的毒性预测，可以使用特定的SMARTS模式
    toxic_substructures = ['[N+]', '[O-]', '[S+]', '[P+]', '[As]', '[Se]', '[Te]']
    for substruct in toxic_substructures:
        pattern = Chem.MolFromSmarts(substruct)
        if mol.HasSubstructMatch(pattern):
            return True
    return False

In [ ]:
# 计算分子的额外属性
def calculate_additional_properties(mol):
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    num_aromatic_rings = Chem.rdMolDescriptors.CalcNumAromaticRings(mol)
    num_aliphatic_rings = Chem.rdMolDescriptors.CalcNumAliphaticRings(mol)
    num_non_h_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() > 1)
    num_non_c_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() != 6)
    non_c_ratio = num_non_c_atoms / num_non_h_atoms if num_non_h_atoms > 0 else 0
    return logp, hbd, hba, num_aromatic_rings, num_aliphatic_rings, non_c_ratio


In [ ]:
# 将分子转换为图数据
def mol_to_graph(mol):
    atoms = mol.GetAtoms()
    atom_features = []
    for atom in atoms:
        atom_features.append([atom.GetAtomicNum()])
    x = torch.tensor(atom_features, dtype=torch.float)
    edge_index = []
    bonds = mol.GetBonds()
    for bond in bonds:
        edge_index.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
        edge_index.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index)
    data.batch = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [ ]:
# 筛选并保存符合条件的分子
optimized_results = []
for mol in optimized_molecules:
    try:
        data = mol_to_graph(mol)
        with torch.no_grad():
            pred_property = model(data).item()
        # 反标准化
        pred_property = pred_property * std + mean

        smiles = Chem.MolToSmiles(mol)
        qed_score = QED.qed(mol)
        lipinski_pass = lipinski_rule_of_five(mol)
        cns_pass = cns_drug_likeness(mol)
        sa_score = synthetic_accessibility(mol)
        toxic = toxicity_prediction(mol)
        logp, hbd, hba, num_aromatic_rings, num_aliphatic_rings, non_c_ratio = calculate_additional_properties(mol)

        # 根据条件筛选分子
        if pred_property >= desired_value and qed_score > 0.7 and lipinski_pass and cns_pass and not toxic:
            optimized_results.append({
                "smiles": smiles,
                "predicted_property": pred_property,
                "QED": qed_score,
                "Lipinski": lipinski_pass,
                "CNS": cns_pass,
                "SA_Score": sa_score,
                "Toxic": toxic,
                "LogP": logp,
                "HBD": hbd,
                "HBA": hba,
                "Aromatic Rings": num_aromatic_rings,
                "Aliphatic Rings": num_aliphatic_rings,
                "Non-C Ratio": non_c_ratio
            })
    except Exception as e:
        # 跳过无法处理的分子
        continue

In [ ]:
# 保存优化后的分子为CSV文件
df = pd.DataFrame(optimized_results)
output_file = DATA / "optimized_drug_like_molecules_with_properties.csv"
df.to_csv(output_file, index=False)
print(f"Optimized molecules saved to: {output_file}")